In [109]:
import math
import numpy as np

In [110]:
def number_of_stubs (nodes_per_prob):
    stubs = 0
    for i in range(0, len(nodes_per_prob)):
        stubs += (i + 1) * nodes_per_prob[i]
    return stubs

def polylogarithm (s: float, z: float, precision=100) -> float:
    power_sum = 0
    for k in range(1, precision):
        power_sum += math.pow(z, k) / math.pow(k, s)
    return power_sum

def poisson (k: int, z: float) -> float:
    return ((math.pow(z, k)) * (math.exp(-z))) / math.factorial(k)

def powerlaw (k: int, gamma: float, kappa: float):
    return (math.pow(k, -gamma) * math.exp(-k / kappa)) / polylogarithm(gamma, math.exp(-1 / kappa))

def generate_poisson_sequence (z: int, population: int, max_degree: int):
    probabilities = [poisson(k, z) for k in range(1, max_degree)]  # Poisson Distribution
    
    nodes_per_prob = [math.ceil(p * population) for p in probabilities]
    print(f'Stubs Before = {number_of_stubs(nodes_per_prob)}')
    print(f'Nodes left to populate = {population - np.sum(nodes_per_prob)}')
    nodes_left = population - np.sum(nodes_per_prob)
    average_degree = round(np.sum([k * poisson(k, z) for k in range(1, max_degree)]))  # Poisson Distribution
    while nodes_left > 0:
        if (nodes_left == 1):
            if (number_of_stubs(nodes_per_prob) + average_degree) % 2 == 0:
                # No stubs left over
                nodes_per_prob[average_degree - 1] += 1
            else:
                # 1 Stub left over, add 1 to average degree
                nodes_per_prob[average_degree] += 1
        else:
            nodes_per_prob[average_degree - 1] += 1 # Add the node into the distribution
        nodes_left -= 1
        
    return nodes_per_prob

def generate_powerlaw_sequence (kappa: int, population: int):
    probabilities = [powerlaw(k, gamma=2, kappa=kappa) for k in range(1, kappa)]  # Power-Law Distribution
    
    nodes_per_prob = [math.ceil(p * population) for p in probabilities]
    nodes_left = population - np.sum(nodes_per_prob)
    average_degree = round(np.sum([k * powerlaw(k, gamma=2, kappa=kappa) for k in range(1, kappa)]))  # Power-Law Distribution
    print(f'Nodes Left: {nodes_left}')
    while nodes_left > 0:
        if (nodes_left == 1):
            if (number_of_stubs(nodes_per_prob) + average_degree) % 2 == 0:
                # No stubs left over
                nodes_per_prob[average_degree - 1] += 1
            else:
                # 1 Stub left over, add 1 to average degree
                nodes_per_prob[average_degree] += 1
        else:
            nodes_per_prob[average_degree - 1] += 1 # Add the node into the distribution
        nodes_left -= 1
        
    return nodes_per_prob

In [116]:
z = 4
kappa = 15
population = 1500
max_degree = 15

# degree_sequence = generate_poisson_sequence(z, population, max_degree)
degree_sequence = generate_powerlaw_sequence(kappa, population)

for k in range(1, max_degree):
    print(f'{k}: {degree_sequence[k - 1]}')

Nodes Left: 6
1: 1005
2: 241
3: 98
4: 52
5: 31
6: 20
7: 14
8: 10
9: 8
10: 6
11: 5
12: 4
13: 3
14: 3


In [118]:
# Check number of stubs is even
np.sum([k * degree_sequence[k - 1] for k in range(1, max_degree)]) % 2 == 0

True

In [119]:
# Check number of nodes is equal to the population
np.sum(degree_sequence) == population

True